## Setup

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
from supabase import create_client, Client
import pandas as pd
import dotenv
import os
import uuid
from datetime import datetime, timezone
import requests
from urllib.parse import urlparse, parse_qs



SERVICE_ACCOUNT_KEY = "./../serviceAccountKey.json"
FILE_PATH = "./../Documentation\SETS Durbar 2025 - Secretaries Bio Data (Responses) (2).xlsx"

dotenv.load_dotenv()

True

In [2]:
cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
firebase_admin.initialize_app(cred)
db = firestore.client()
supabase = create_client(os.getenv('EXPO_PUBLIC_SUPABASE_URL'), os.getenv('EXPO_PUBLIC_SUPABASE_ANON_KEY'))

In [46]:
df = pd.read_excel(FILE_PATH)

## Preprocess

In [47]:
df.head()

,Timestamp,Email address,Full Name,Date of Birth,Sex,Photograph,Spouse's Name,Wedding Anniversary,Phone,Email,...,Business Address,Business Website,About your business,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Phone,T Shirt Size,Shirt Size,Meal Preference,Blood Group
0,2024-12-06 15:01:05.435,jackdeepan14@googlemail.com,Deepan P,1990-06-14,Male,https://drive.google.com/open?id=13WWgh0j02LNE...,Uma S,27/10,9884958988,deepan_90@live.com,...,"2nd floor ,RR tower 3,industrial estate Guindy",NaN,Im a working professional and not a business man,9884958988,Spouse,9884958988,XXL,42,Non-Vegetarian,NaN
1,2024-12-06 15:07:10.859,rrssbl@gmail.com,Ramasubramanian R,1948-10-31,Male,https://drive.google.com/open?id=1iGyxH_jLkL2c...,R Chandra,02/02,9840356286,rrssbl@gmail.com,...,"Sundaram Medical Devices , CIT colony Mylapore",NaN,Manufacturing Medical Devices,Sriram Ramasubramanian,Son,9940666880,XL,42,Vegetarian,NaN
2,2024-12-09 16:30:46.361,arun@escrowtech.in,Arun Mahadevan,1976-09-16,Male,https://drive.google.com/open?id=1sGPpsSej-9U-...,Dr. CS Ramya,13/09,9841162367,arunmahadevan@hotmail.com,...,"C2-A, industrial Estate, Guindy 600032",https://escrowtech.in/,"EscrowTech India, an Indo-US joint venture who...",Sudhir Ravindran,Friend,9383152137,L,42,Vegetarian,NaN
3,2024-12-14 13:05:33.811,ravi.padmanaban@gmail.com,RAVI PADMANABAN,1978-03-22,Male,https://drive.google.com/open?id=1kibRraxwCQBn...,Anusha Sivaprakasam,05/09,9003199405,ravi.padmanaban@gmail.com,...,"CD8, Vikas Hillview Apt, 159 Dargah Road, Zami...",NaN,Software consultant providing services to few ...,Anusha Sivaprakasam,Wife,8695982100,XL,42,Vegetarian,NaN
4,2024-12-14 13:11:48.403,sanjusuresh9715@gmail.com,Sarjana Suresh,1997-09-15,Female,https://drive.google.com/open?id=1OW60gLuH-pBf...,Praveen Kumar,03/09,9952789379,sanjusuresh@mgrjanaki.ac.in,...,"11&13 Sathyabama Maligai, Durgabai Deshmukh ro...",https://www.mgrjanaki.ac.in/,I am a special educator. I specialise in Engli...,Shylaja,Mother,8939275475,XXL,42,Non-Vegetarian,NaN


In [48]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_').str.replace("'", "")
df.rename(columns={
    "if_self-employed_:_business_name_/_if_salaried_:_company_name": "company_name",
    "designation": "sets_designation",
    "role_in_business_/_designation_in_company": "designation",
    "full_name": "name"}, inplace=True)
df.drop(columns=['timestamp'], inplace=True)

In [49]:
df["name_lower"] = df["name"].str.lower()
df["company_name_lower"] = df["company_name"].str.lower()
df["club_name_lower"] = df["club_name"].str.lower()
df["priority"] = 0
df['role'] = "member"
df["support"] = False
df.fillna("", inplace=True)

In [50]:
def convert_timestamp_to_string(timestamp):
    
    if pd.isna(timestamp):
        return ""
    
    if isinstance(timestamp, datetime):
        if timestamp.tzinfo is None:  # Check if the timestamp is naive
            timestamp = timestamp.replace(tzinfo=timezone.utc)
        else:
            timestamp = timestamp.astimezone(timezone.utc)
    formatted_string = timestamp.strftime('%Y-%m-%d %H:%M:%S%z')
    if formatted_string.endswith("+0000"):
        formatted_string = formatted_string[:-2] + ":00"
    return formatted_string

In [51]:
def transform_date(value):
    # Check if the value is in DD/MM format or full date format
    if not value:
        return ""
    
    try:
        # Attempt to parse full date
        date = pd.to_datetime(value, format='%d/%m/%Y')
        return date.strftime('%d/%m/%Y')
    except ValueError:
        # If ValueError occurs, assume the value is in DD/MM format
        current_year = datetime.now().year
        return f"{value}/{current_year}"

In [52]:
df.columns

Index(['email_address', 'name', 'date_of_birth', 'sex', 'photograph',
       'spouses_name', 'wedding_anniversary', 'phone', 'email', 'club_name',
       'rotarian_since', 'rotary_foundation_title', 'residential_address',
       'company_name', 'designation', 'type_of_business', 'business_address',
       'business_website', 'about_your_business', 'emergency_contact_name',
       'emergency_contact_relationship', 'emergency_contact_phone',
       't_shirt_size', 'shirt_size', 'meal_preference', 'blood_group',
       'name_lower', 'company_name_lower', 'club_name_lower', 'priority',
       'role', 'support'],
      dtype='object')

In [53]:
datetime_columns = ['date_of_birth', 'wedding_anniversary']

In [54]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
# df['wedding_anniversary'] = pd.to_datetime(df['wedding_anniversary'], errors='coerce')
df['wedding_anniversary'] = df['wedding_anniversary'].apply(transform_date)
df['wedding_anniversary'] = pd.to_datetime(df['wedding_anniversary'], errors='coerce', dayfirst=True)


In [55]:
for col in datetime_columns:
    df[col] = df[col].dt.tz_localize("UTC", ambiguous='NaT', nonexistent='NaT')

In [57]:
for col in datetime_columns:
    df[col] = df[col].apply(convert_timestamp_to_string)

In [60]:
data = df.to_dict(orient='records')

## Upload Image + Return Public URL

In [15]:
def download_file_from_google_drive(uid, photo_url):       
    parsed_url = urlparse(photo_url)
    query_params = parse_qs(parsed_url.query)
    file_id = query_params['id'][0]
    
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": file_id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    return save_response_content(uid, response)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None

def save_response_content(uid, response):   
    file_name = f"{uid}.png"  # Assuming the image format is JPEG
    file_content = response.content
    
    try:
        responseSupabase = supabase.storage.from_("members").upload(file_name, file_content)
        public_url = supabase.storage.from_("members").get_public_url(file_name)
        return public_url
    except Exception as e:
        print(e)


## Upsert Users

- Check for email existence in firebase auth, if it exists then update record in firestore
- If email does not exist then create new user in firebase auth and then create new record in firestore with doc id = user.uid

In [21]:
for index, user_data in enumerate(data):
    email = user_data.get("email")
    if not email:
        print(f"Skipping record without email: {user_data}")
        continue
    
    try:
        # Check if user exists in Firebase Auth
        user = auth.get_user_by_email(email)
        print(f"({index}/{len(data)}) User exists: {email} (UID: {user.uid}). Updating record")
        
        # Update Firestore record
        user_data['id'] = user.uid
        try:
            public_url = download_file_from_google_drive(user_data['id'], user_data['photograph'])
            user_data['photograph'] = public_url
            # Upsert the record into Supabase
            response = supabase.table("members").upsert(user_data).execute()

            if len(response.data) <= 0:
                print(f"Error while upserting record for email {email}: {response}")
            else:
                print(f"Successfully upserted record for email {email}")

        except Exception as e:
            print(f"An error occurred while processing email {email}: {str(e)}")
        
    except auth.UserNotFoundError:
        print(f"({index}/{len(data)}) User does not exist: {email}")
        
        # Create new user in Firebase Auth
        new_user = auth.create_user(email=email, password=email.split('@')[0] + '_password')
        print(f"Created new user: {email} (UID: {new_user.uid})")
        
        # Create new Firestore record
        user_data['id'] = user.uid
        try:
            public_url = download_file_from_google_drive(user_data['id'], user_data['photograph'])
            user_data['photograph'] = public_url
            
            response = supabase.table("members").upsert(user_data).execute()

            if len(response.data) <= 0:
                print(f"Error while upserting record for email {email}: {response}")
            else:
                print(f"Successfully upserted record for email {email}")

        except Exception as e:
            print(f"An error occurred while processing email {email}: {str(e)}")

(0/58) User exists: deepan_90@live.com (UID: pjgqrIvzbhXBAHx0WM1qxT4M1va2). Updating record
An error occurred while processing email deepan_90@live.com: 'id'
(1/58) User exists: rrssbl@gmail.com (UID: Itl7GUp6bQW5jOuGVfHq2op9col2). Updating record
An error occurred while processing email rrssbl@gmail.com: 'id'
(2/58) User exists: arunmahadevan@hotmail.com (UID: wYCW25rYsbRqtANt7gAc7vjqNxh2). Updating record
An error occurred while processing email arunmahadevan@hotmail.com: 'id'
(3/58) User exists: ravi.padmanaban@gmail.com (UID: Dz2YEaWJx5Xth7dfJVkJV9AqBWs2). Updating record
An error occurred while processing email ravi.padmanaban@gmail.com: 'id'
(4/58) User exists: sanjusuresh@mgrjanaki.ac.in (UID: 48ywCUWkqnP7IwVM3StBztjZ1Ar1). Updating record
{'statusCode': 400, 'error': 'Duplicate', 'message': 'The resource already exists'}
Successfully upserted record for email sanjusuresh@mgrjanaki.ac.in
(5/58) User exists: umamahes03@gmail.com (UID: o7TUgcOfifPfHHBAOqlGfyxXXyk2). Updating recor

In [63]:
for index, user_data in enumerate(data):
    # Check if user exists in Firebase Auth
    email = user_data.get("email")
    user = auth.get_user_by_email(email)
    print(f"({index}/{len(data)}) User exists: {email} (UID: {user.uid}). Updating record")
    
    # Update Firestore record
    user_data['id'] = user.uid
    record = {'id': user.uid, 'date_of_birth': user_data['date_of_birth'], 'wedding_anniversary': user_data['wedding_anniversary']}
    try:
        response = supabase.table("members").upsert(record).execute()

        if len(response.data) <= 0:
            print(f"Error while upserting record for email {email}: {response}")
        else:
            print(f"Successfully upserted record for email {email}")

    except Exception as e:
        print(f"An error occurred while processing email {email}: {str(e)}")

(0/58) User exists: deepan_90@live.com (UID: pjgqrIvzbhXBAHx0WM1qxT4M1va2). Updating record
Successfully upserted record for email deepan_90@live.com
(1/58) User exists: rrssbl@gmail.com (UID: Itl7GUp6bQW5jOuGVfHq2op9col2). Updating record
Successfully upserted record for email rrssbl@gmail.com
(2/58) User exists: arunmahadevan@hotmail.com (UID: wYCW25rYsbRqtANt7gAc7vjqNxh2). Updating record
Successfully upserted record for email arunmahadevan@hotmail.com
(3/58) User exists: ravi.padmanaban@gmail.com (UID: Dz2YEaWJx5Xth7dfJVkJV9AqBWs2). Updating record
Successfully upserted record for email ravi.padmanaban@gmail.com
(4/58) User exists: sanjusuresh@mgrjanaki.ac.in (UID: 48ywCUWkqnP7IwVM3StBztjZ1Ar1). Updating record
Successfully upserted record for email sanjusuresh@mgrjanaki.ac.in
(5/58) User exists: umamahes03@gmail.com (UID: o7TUgcOfifPfHHBAOqlGfyxXXyk2). Updating record
Successfully upserted record for email umamahes03@gmail.com
(6/58) User exists: vichandrasekar@gmail.com (UID: Ln4